<a href="https://colab.research.google.com/github/nyanta012/demo/blob/main/langchain_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install langchain==0.0.157 openai==0.27.2 google-search-results==2.4.2

In [4]:
from langchain.agents import load_tools
from langchain.agents import Tool
from langchain.agents import initialize_agent
from langchain import PromptTemplate, LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback
from langchain.agents import AgentType

import os
os.environ["SERPAPI_API_KEY"] = "***"
os.environ["OPENAI_API_KEY"] = "***"

In [7]:
# 使用するモデルを定義
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

In [10]:
# google 検索を行う機能を追加
from langchain import SerpAPIWrapper

search = SerpAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events.",
    )
]

In [11]:
# 単純に入力を受け取ってモデルに入れるプロンプト
prompt = PromptTemplate(input_variables=["query"], template="{query}")

# chainの定義
llm_chain = LLMChain(llm=llm, prompt=prompt)

# toolsに追加して、モデルに使って貰う
tools.append(
    Tool(
        name="Language Model",
        func=llm_chain.run,
        description="use this tool for general purposes like answer question and output formatting",
    )
)

In [12]:
# 計算機能を追加
from langchain.chains import LLMMathChain
llm_math = LLMMathChain(llm=llm)

tools.append(Tool(
        name="Calculator",
        func=llm_math.run,
        description="Useful for when you need to answer questions about calculation.",
    ))

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm_math/base.py:50: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [13]:
# agentの定義
agent = initialize_agent(
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
)

In [14]:
print(agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Search: useful for when you need to answer questions about current events.
Language Model: use this tool for general purposes like answer question and output formatting
Calculator: Useful for when you need to answer questions about calculation.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Language Model, Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [15]:
with get_openai_callback() as cb:
    response = agent.run("2023年WBCの優勝国はどこでしたか？日本語で答えてください")



> Entering new AgentExecutor chain...
I need to search for the answer to this question.
Action: Search
Action Input: "2023 WBC 優勝国"
Observation: Japan national baseball team
Thought:I need to translate the answer into Japanese.
Action: Language Model
Action Input: "Japan national baseball team" (translate to Japanese)
Observation: 日本代表野球チーム
Thought:I have the answer in Japanese, but I need to confirm the year.
Action: Search
Action Input: "2023 WBC 優勝国 日本代表野球チーム"
Observation: 侍ジャパンがアメリカを下し3大会ぶり3回目のWBC制覇！ 大谷翔平が胴上げ投手に 3月21日（日本時間3月22日)、『2023 WORLD BASEBALL CLASSIC™』の決勝戦が行われ、侍ジャパンがMLBを代表する選手たちを揃えたアメリカを3対2で下し、7戦全勝で3大会ぶり3回目の優勝を果たした。
Thought:I now know the final answer.
Final Answer: 2023年のWBCの優勝国は日本代表野球チームでした。

> Finished chain.


In [17]:
response

'2023年のWBCの優勝国は日本代表野球チームでした。'

In [18]:
cb

Tokens Used: 1387
	Prompt Tokens: 1237
	Completion Tokens: 150
Successful Requests: 5
Total Cost (USD): $0.0027740000000000004

In [19]:
cb.total_cost * 136

0.37726400000000004